In [3]:
import os  # a conflict in my mac system
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

conv_base = keras.applications.mobilenet.MobileNet(input_shape=(224,224,3), alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=False, weights='imagenet')

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
# this "freezes"
conv_base.trainable = False
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               12845312  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 16,076,746
Trainable params: 12,847,882
Non-trainable params: 3,228,864
_________________________________________________________________


In [4]:

train_dir = "/Users/lilingxiao/Downloads/state-farm-distracted-driver-detection/imgs/train"
test_dir = "/Users/lilingxiao/Downloads/state-farm-distracted-driver-detection/imgs/val"

train_data_generator = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
test_data_generator = ImageDataGenerator(rescale=1. / 255)
train_generator = train_data_generator.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=40, class_mode='categorical')
test_generator = test_data_generator.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=40,
    class_mode='categorical')


Found 18414 images belonging to 10 classes.
Found 4010 images belonging to 10 classes.


In [5]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=['acc'])


In [19]:
model.load_weights('Distracted_mobilenet.h5', by_name=True)
history = model.fit_generator(
    train_generator,
    steps_per_epoch=400,
    epochs=2,
    validation_data=test_generator,
    validation_steps=80)

model.save_weights('Distracted_mobilenet.h5')


Epoch 1/2
400/400 [==============================] - 299s 748ms/step - loss: 0.5789 - acc: 0.8190 - val_loss: 1.3227 - val_acc: 0.7122
Epoch 2/2
400/400 [==============================] - 299s 746ms/step - loss: 0.5552 - acc: 0.8218 - val_loss: 1.2389 - val_acc: 0.7303


In [7]:
model.save_weights('Distracted_mobilenet.h5')

In [20]:
model.save('Distracted_mobilenet_full.h5')